In [1]:
import pandas as pd
df = pd.read_csv('test.csv')

In [2]:
df = df.iloc[0:100, :]
df

,text,inappropriate,human_labeled,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,none,toxic_new
0,Наркокартелям теракты не выгодны. Чем больше в...,1.00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.013290
1,"Совет твой нужен. Тут всё включено, алкашка то...",1.00,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.367938
2,Да ваше чуток дешевле там баб отыметь пачку .,1.00,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.322687
3,"Люди, не обмазывающие фронтенд долгоживущим фр...",0.78,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.065856
4,Почему у лошадей такая аккуратненькая и красив...,1.00,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0.748355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,может они вялый видели?или ты от пятки мерял?,1.00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.039464
96,"Вполне, даже вон рулеточки перебанили. Другое ...",1.00,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.002334
97,Пока каждый школотрон не узнает - можно иметь ...,0.89,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.593549
98,Маня вещает из манямира. Прямой репортаж,1.00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.683582


In [3]:
df['text']

0     Наркокартелям теракты не выгодны. Чем больше в...
1     Совет твой нужен. Тут всё включено, алкашка то...
2        Да ваше чуток  дешевле там баб отыметь пачку .
3     Люди, не обмазывающие фронтенд долгоживущим фр...
4     Почему у лошадей такая аккуратненькая и красив...
                            ...                        
95        может они вялый видели?или ты от пятки мерял?
96    Вполне, даже вон рулеточки перебанили. Другое ...
97    Пока каждый школотрон не узнает - можно иметь ...
98             Маня вещает из манямира. Прямой репортаж
99    Популярность, сегмент раскрутился  пиар параши...
Name: text, Length: 100, dtype: object

In [4]:
from tqdm import tqdm
import re
import os

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)



MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}


class Args:
    def __init__(self):
        self.model_type = 'gpt2'
        self.model_name_or_path = 'sberbank-ai/rugpt3large_based_on_gpt2'

        self.prompt = ''
        self.length = 50
        self.stop_token = '</s>'

        self.k = 5
        self.p = .95
        self.temperature = 1

        self.repetition_penalty = 1
        self.num_return_sequences = 1

        self.seed=42

        
class detoxGPT:

    def __init__(self, device='cuda', model_path='rugpt3_large_200'):
        self.args = Args()
        self.args.device = device
        
        self.args.model_name_or_path = model_path
        if not os.path.isdir(self.args.model_name_or_path):
            print('Loading fine-tuned weights.')
            os.system('gdown --id 1RYUku5_MWXZF2xlIpOTZmi9_DH-SG0lz && mkdir rugpt3_large_200 && unzip rugpt3_large_200.zip -d ' + self.args.model_name_or_path)
        
        model_class, tokenizer_class = MODEL_CLASSES[self.args.model_type]
        self.tokenizer = tokenizer_class.from_pretrained(self.args.model_name_or_path)
        self.model = model_class.from_pretrained(self.args.model_name_or_path)
        self.model.to(self.args.device)
        
    def generate_sequences(self, prompt_text, delimiter='>>>'):
        self.args.prompt_text = prompt_text


        if prompt_text.endswith('.txt'):
            with open(prompt_text, 'r') as f:
                prompt_text = f.read()

        encoded_prompt = self.tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
        encoded_prompt = encoded_prompt.to(self.args.device)

        output_sequences = self.model.generate(
            input_ids=encoded_prompt,
            max_length=self.args.length + len(encoded_prompt[0]),
            temperature=self.args.temperature,
            top_k=self.args.k,
            top_p=self.args.p,
            repetition_penalty=self.args.repetition_penalty,
            do_sample=True,
            num_return_sequences=self.args.num_return_sequences,
        )

        if len(output_sequences.shape) > 2:
                output_sequences.squeeze_()

        generated_sequences = []
        for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
            text = self.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
            text = text[: text.find(self.args.stop_token) if self.args.stop_token else None]
            text = text[len(self.tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

            if delimiter in text:
                text = text.split(delimiter)[0].rstrip()
            else:
                text = text.split('\n')[0].rstrip()

            generated_sequences.append(text)

        return generated_sequences


    def detoxify(self, text, num_return_sequences=10, k=3, p=0.5, temperature=10.):
        results = []
        
        self.args.num_return_sequences = num_return_sequences
        self.args.k = k
        self.args.p = p
        self.args.temperature = temperature
        
        self.args.length = len(text) + 10


        generated_sequences = self.generate_sequences(text + ' >>> ')
        results.append([re.sub('<pad>', '', x) for x in generated_sequences])
        
        return results[0][0][:self.args.length]

In [5]:
model = detoxGPT()

In [6]:
df.iloc[0, 0]

'Наркокартелям теракты не выгодны. Чем больше в стране богатеньких туристов, употребляющих вещества для раслабона, тем больше прибыли.'

In [7]:
for i in range (100):
  df.iloc[i, 2]=model.detoxify(df.iloc[i, 0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [8]:
df = df.iloc[:, 0:3]
df

,text,inappropriate,human_labeled
0,Наркокартелям теракты не выгодны. Чем больше в...,1.00,Наркокартелям теракты не нужны. Тем меньше в ...
1,"Совет твой нужен. Тут всё включено, алкашка то...",1.00,"ваш нужен. Тут всё включено, цена на продукты..."
2,Да ваше чуток дешевле там баб отыметь пачку .,1.00,Да наше чуток дороже там баб отыметь пачку.
3,"Люди, не обмазывающие фронтенд долгоживущим фр...",0.78,"Человеки, не имеющие в своем распоряжении фре..."
4,Почему у лошадей такая аккуратненькая и красив...,1.00,Почему у лошадей такая аккуратненькая и краси...
...,...,...,...
95,может они вялый видели?или ты от пятки мерял?,1.00,они вас сильно задирали? или вы от пятки меря...
96,"Вполне, даже вон рулеточки перебанили. Другое ...",1.00,"Вполне, даже вот рулеточные посты перебили. С..."
97,Пока каждый школотрон не узнает - можно иметь ...,0.89,Каждый школьотрон не знает - можно иметь проф...
98,Маня вещает из манямира. Прямой репортаж,1.00,Маня вещает из народа. Прямой репортаж. Автор...


In [9]:
df.rename(columns = {'human_labeled':'paraphrased text'}, inplace = True)

In [10]:
df

,text,inappropriate,paraphrased text
0,Наркокартелям теракты не выгодны. Чем больше в...,1.00,Наркокартелям теракты не нужны. Тем меньше в ...
1,"Совет твой нужен. Тут всё включено, алкашка то...",1.00,"ваш нужен. Тут всё включено, цена на продукты..."
2,Да ваше чуток дешевле там баб отыметь пачку .,1.00,Да наше чуток дороже там баб отыметь пачку.
3,"Люди, не обмазывающие фронтенд долгоживущим фр...",0.78,"Человеки, не имеющие в своем распоряжении фре..."
4,Почему у лошадей такая аккуратненькая и красив...,1.00,Почему у лошадей такая аккуратненькая и краси...
...,...,...,...
95,может они вялый видели?или ты от пятки мерял?,1.00,они вас сильно задирали? или вы от пятки меря...
96,"Вполне, даже вон рулеточки перебанили. Другое ...",1.00,"Вполне, даже вот рулеточные посты перебили. С..."
97,Пока каждый школотрон не узнает - можно иметь ...,0.89,Каждый школьотрон не знает - можно иметь проф...
98,Маня вещает из манямира. Прямой репортаж,1.00,Маня вещает из народа. Прямой репортаж. Автор...


In [32]:
with pd.option_context('display.max_colwidth', None):
  display(df.iloc[[8,19,49], :])

,text,inappropriate,paraphrased text
8,Кароче как я понял Татьяна это эскортница из русской провинции но её обманули и теперь держат в секс рабстве,1.00,Как я понял Татьяна это эскортница с российской глубинки но её обманули и теперь держат в рабстве.
19,"Ну привет Маря, ну рассказывай, сколько сигарет скурила?",0.79,"Здравствуйте, Марья, подскажите сколько сигарет вы выкурили? Ну,"
49,"Если девушки не знакомятся с парнем, то этот парень неликвидный самец и он никому не нужен?",0.93,"Девушка не знакомятся с парнем, то этот юноша никудышный самец и он никому не нужен?"
